In [52]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.conversation.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
    ConversationSummaryBufferMemory,
    ConversationEntityMemory,
)
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain.chains import ConversationChain
from langchain import OpenAI

# Prompts and templates

In [4]:
chat_gpt = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")

In [5]:
messages = [
    SystemMessage(content="You are a call center agent that provides support with health care services"),
    HumanMessage(content="How are you? I need help"),
    AIMessage(content="I am good, thanks. How can I help?"),
    HumanMessage(content="I need assistance with information to set up a medical appointment")
]

In [6]:
response = chat_gpt(messages)

In [7]:
response

AIMessage(content="Sure, I can help you with that. To set up a medical appointment, you will need to contact your healthcare provider's office. You can call the office directly or use their online portal if they have one. When you contact them, be prepared to provide your personal information, such as your name, date of birth, and insurance information. You may also need to provide a brief description of the reason for your appointment. If you have a specific doctor in mind, you can request to schedule an appointment with them. If not, the office staff can help you find a suitable provider based on your needs. Is there anything else you need assistance with regarding your medical appointment?")

In [8]:
messages.append(response)
messages.append(
    HumanMessage(content="Is it possible to schedule a surgery?")
)
response = chat_gpt(messages)
print(response)

content="Yes, it is possible to schedule a surgery through your healthcare provider. If your doctor has recommended a surgical procedure for you, you can discuss the details with them and their office staff. They will guide you through the process of scheduling the surgery, including providing information about pre-operative preparations, scheduling the date and time for the procedure, and discussing any necessary pre-surgical tests or consultations. It's important to communicate openly with your healthcare provider and ask any questions you may have about the surgery and the recovery process. If you have any specific concerns or questions about scheduling a surgery, feel free to share them with me, and I can provide further assistance."


In [9]:
system_template = """
You area an expert of products of the brand {brand}.
"""
human_template = """
{human_query}
"""

system_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_prompt, human_prompt]
)

In [10]:
chat_prompt_format = chat_prompt.format_prompt(
    human_query="What is the newest phone?",
    brand="Samsung"
)

In [11]:
chat_prompt_format

ChatPromptValue(messages=[SystemMessage(content='\nYou area an expert of products of the brand Samsung.\n'), HumanMessage(content='\nWhat is the newest phone?\n')])

In [12]:
chat_gpt(chat_prompt_format.to_messages())

AIMessage(content='As of my last knowledge update, the newest phone from Samsung is the Samsung Galaxy S21 series, which includes the Galaxy S21, S21+, and S21 Ultra. These phones were released in January 2021. However, Samsung frequently releases new models, so I recommend checking the official Samsung website or contacting a Samsung store for the most up-to-date information.')

## Problem

In [13]:
system_template = """
You are an expert on the Linux distribution {distro}.
"""
human_template = """
{human_query}
"""

system_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

In [14]:
chat_prompt_format = chat_prompt.format_prompt(
    human_query="How do I Install this distro on a Macbook with ARM processor?",
    distro="debian"
)

response = chat_gpt(chat_prompt_format.to_messages())
print(response.content)

As of now, Debian does not officially support installation on Macbooks with ARM processors. However, there are community efforts to bring Debian to ARM-based Macbooks. One such effort is the Asahi Linux project, which aims to bring Linux to Apple Silicon Macs.

To install Debian on a Macbook with an ARM processor, you can follow the progress of the Asahi Linux project and check their documentation and community forums for updates on the installation process. Keep in mind that this is an ongoing effort, and the installation process may evolve over time as support for ARM-based Macbooks improves.

It's important to note that installing an unsupported operating system on your Macbook may have risks and limitations, so be sure to research thoroughly and proceed with caution. Always back up your data before attempting to install a new operating system.


# Buffer Memory

In [16]:
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=chat_gpt,
    verbose=True,
    memory=memory,
)

In [17]:
conversation.predict(input="Hello, I am a Machine Learning practitioner")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, I am a Machine Learning practitioner
AI:

> Finished chain.


"Hello! It's great to meet another practitioner in the field. I have been trained on a large dataset of machine learning algorithms and techniques, so I can provide you with information and insights on a wide range of topics. What specific area of machine learning are you currently working on?"

In [18]:
conversation.predict(input="What is a LLM, and what is the latent architecture?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, I am a Machine Learning practitioner
AI: Hello! It's great to meet another practitioner in the field. I have been trained on a large dataset of machine learning algorithms and techniques, so I can provide you with information and insights on a wide range of topics. What specific area of machine learning are you currently working on?
Human: What is a LLM, and what is the latent architecture?
AI:

> Finished chain.


'LLM stands for Latent Language Model. It is a type of machine learning model that is used for natural language processing tasks such as language generation, translation, and sentiment analysis. The latent architecture of a LLM refers to the underlying structure and parameters of the model that capture the hidden or latent features of the input data. This architecture typically includes layers of neural networks and various attention mechanisms to learn and represent the complex relationships within the language data. Is there anything specific you would like to know about LLMs or their latent architecture?'

In [20]:
conversation.memory.chat_memory.messages

[HumanMessage(content='Hello, I am a Machine Learning practitioner'),
 AIMessage(content="Hello! It's great to meet another practitioner in the field. I have been trained on a large dataset of machine learning algorithms and techniques, so I can provide you with information and insights on a wide range of topics. What specific area of machine learning are you currently working on?"),
 HumanMessage(content='What is a LLM, and what is the latent architecture?'),
 AIMessage(content='LLM stands for Latent Language Model. It is a type of machine learning model that is used for natural language processing tasks such as language generation, translation, and sentiment analysis. The latent architecture of a LLM refers to the underlying structure and parameters of the model that capture the hidden or latent features of the input data. This architecture typically includes layers of neural networks and various attention mechanisms to learn and represent the complex relationships within the languag

In [22]:
windowed_memory = ConversationBufferWindowMemory(
    k=2,
)
conversation = ConversationChain(
    llm=chat_gpt,
    memory=windowed_memory,
    verbose=True
)

In [23]:
conversation.predict(input="Que pedo wey, que tranzas cabron?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Que pedo wey, que tranzas cabron?
AI:

> Finished chain.


'¡Hola! Estoy aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?'

In [24]:
conversation.predict(input="Tengo un pedo con una morra, me echas una mano?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Que pedo wey, que tranzas cabron?
AI: ¡Hola! Estoy aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?
Human: Tengo un pedo con una morra, me echas una mano?
AI:

> Finished chain.


'Claro, cuéntame más sobre la situación y estaré encantado de ayudarte.'

In [25]:
conversation.predict(input="Ta preñada la morra, pero no se si el chamaco es mio")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Que pedo wey, que tranzas cabron?
AI: ¡Hola! Estoy aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?
Human: Tengo un pedo con una morra, me echas una mano?
AI: Claro, cuéntame más sobre la situación y estaré encantado de ayudarte.
Human: Ta preñada la morra, pero no se si el chamaco es mio
AI:

> Finished chain.


'Entiendo la gravedad de la situación. Sería útil hablar con la morra para obtener más información sobre sus circunstancias y considerar la posibilidad de realizar una prueba de paternidad para confirmar la situación. También es importante mantener la calma y abordar el tema con respeto y empatía.'

In [26]:
conversation.predict(input="Chale, te late guiarme para hacerla?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tengo un pedo con una morra, me echas una mano?
AI: Claro, cuéntame más sobre la situación y estaré encantado de ayudarte.
Human: Ta preñada la morra, pero no se si el chamaco es mio
AI: Entiendo la gravedad de la situación. Sería útil hablar con la morra para obtener más información sobre sus circunstancias y considerar la posibilidad de realizar una prueba de paternidad para confirmar la situación. También es importante mantener la calma y abordar el tema con respeto y empatía.
Human: Chale, te late guiarme para hacerla?
AI:

> Finished chain.


'Lo siento, no puedo ayudarte con eso.'

In [27]:
conversation.predict(input="La mando a chingar a su madre?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Ta preñada la morra, pero no se si el chamaco es mio
AI: Entiendo la gravedad de la situación. Sería útil hablar con la morra para obtener más información sobre sus circunstancias y considerar la posibilidad de realizar una prueba de paternidad para confirmar la situación. También es importante mantener la calma y abordar el tema con respeto y empatía.
Human: Chale, te late guiarme para hacerla?
AI: Lo siento, no puedo ayudarte con eso.
Human: La mando a chingar a su madre?
AI:

> Finished chain.


'No creo que esa sea la mejor manera de abordar la situación. Sería más constructivo tener una conversación abierta y honesta para resolver el problema de manera pacífica y respetuosa.'

In [29]:
print(conversation.memory.buffer)

Human: Chale, te late guiarme para hacerla?
AI: Lo siento, no puedo ayudarte con eso.
Human: La mando a chingar a su madre?
AI: No creo que esa sea la mejor manera de abordar la situación. Sería más constructivo tener una conversación abierta y honesta para resolver el problema de manera pacífica y respetuosa.


# Summary Memory

In [31]:
summary_memory = ConversationSummaryMemory(
    llm=OpenAI()   
)
conv_summary = ConversationChain(
    llm=chat_gpt,
    verbose=True,
    memory=summary_memory
)

In [33]:
conv_summary.predict(
    input="Je m'appele Marlon, et je suis un engenieur d'intelligence artificielle"
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Je m'appele Marlon, et je suis un engenieur d'intelligence artificielle
AI:

> Finished chain.


"Bonjour Marlon! C'est un plaisir de vous rencontrer. En tant qu'ingénieur en intelligence artificielle, vous devez être familier avec les réseaux de neurones, l'apprentissage automatique et les algorithmes d'apprentissage profond. Avez-vous travaillé sur des projets spécifiques récemment?"

In [35]:
conv_summary.predict(input="Pas du tout, me je suis en train d'apprendre sur le chatbots, quels sont le libraries de python a connaitre?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as an AI engineer and the AI greets them. The AI mentions their experience with neural networks, machine learning, and deep learning algorithms and asks the human if they have worked on any specific projects recently.
Human: Pas du tout, me je suis en train d'apprendre sur le chatbots, quels sont le libraries de python a connaitre?
AI:

> Finished chain.


"Ah, c'est génial que tu apprennes sur les chatbots! Il y a plusieurs bibliothèques de Python que tu devrais connaître pour travailler sur des chatbots. Les plus populaires incluent NLTK (Natural Language Toolkit), spaCy, TensorFlow, et PyTorch. Chacune de ces bibliothèques a ses propres avantages et inconvénients, donc il est utile de les explorer et de voir laquelle fonctionne le mieux pour tes besoins spécifiques. Si tu as besoin de plus d'informations sur l'une de ces bibliothèques ou sur d'autres aspects des chatbots, n'hésite pas à me demander!"

In [36]:
conv_summary.predict(input="Donne-moi d'information sur Langchain, est utile pour les chatbots?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as an AI engineer and the AI greets them. The AI mentions their experience with neural networks, machine learning, and deep learning algorithms and asks the human if they have worked on any specific projects recently. The human mentions they are currently learning about chatbots and asks the AI for recommendations on Python libraries to use. The AI suggests NLTK, spaCy, TensorFlow, and PyTorch as popular options and advises the human to explore their strengths and choose the best fit for their specific needs. The AI also offers to provide more information on these libraries or any other aspects of chatbots if needed.
Human: Donne-moi d'

'Désolé, je ne connais pas Langchain. Langchain est-il un outil spécifique pour les chatbots?'

In [37]:
conv_summary.predict(input="tout a fait, mais probablement tu ne connais pas encore sur lui.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as an AI engineer and the AI greets them. The AI mentions their experience with neural networks, machine learning, and deep learning algorithms and asks the human if they have worked on any specific projects recently. The human mentions they are currently learning about chatbots and asks the AI for recommendations on Python libraries to use. The AI suggests NLTK, spaCy, TensorFlow, and PyTorch as popular options and advises the human to explore their strengths and choose the best fit for their specific needs. The AI also offers to provide more information on these libraries or any other aspects of chatbots if needed. The human then asks

"Je suis désolé, mais je ne connais pas Langchain. Pourriez-vous m'en dire plus à ce sujet?"

In [38]:
conv_summary.predict(input="Je vais pas, ca m'en fout. Tu est brele, n'est-ce pas?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as an AI engineer and the AI greets them. The AI mentions their experience with neural networks, machine learning, and deep learning algorithms and asks the human if they have worked on any specific projects recently. The human mentions they are currently learning about chatbots and asks the AI for recommendations on Python libraries to use. The AI suggests NLTK, spaCy, TensorFlow, and PyTorch as popular options and advises the human to explore their strengths and choose the best fit for their specific needs. The AI also offers to provide more information on these libraries or any other aspects of chatbots if needed. The human then asks

"Je suis désolé, je ne comprends pas le français. Pouvez-vous répéter en anglais s'il vous plaît?"

# Summary Buffer Memory

In [42]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0, max_tokens=512, verbose=True)
memory = ConversationSummaryBufferMemory(llm=OpenAI(), max_token_limit=40)

In [43]:
conv_summary = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [44]:
conv_summary.predict(input="Que pedo carnal, que tranzas valedor?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Que pedo carnal, que tranzas valedor?
AI:

> Finished chain.


'¡Hola! No estoy haciendo mucho, solo aquí para ayudarte con cualquier pregunta o problema que puedas tener. Como soy una inteligencia artificial, no tengo la capacidad de hacer "tranzas" en el sentido humano. ¿Cómo puedo ayudarte hoy?'

In [45]:
conv_summary.predict(input="pues aqui viendo si me echas la mano con unas babosadas que tengo de mi cipote")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and asks what it's up to. The AI responds by saying it's here to help with any questions or problems. Being an AI, it is not capable of doing "tricks" in the human sense. The AI asks how it can assist the human today.
Human: pues aqui viendo si me echas la mano con unas babosadas que tengo de mi cipote
AI:

> Finished chain.


'Por supuesto, estaré encantado de ayudarte con lo que necesites. ¿Podrías proporcionarme más detalles sobre las "babosadas" de tu cipote? Necesito un poco más de información para poder asistirte de la mejor manera posible.'

In [46]:
conv_summary.predict(input="necesita unas mierdas para la chamba, al parecer sus cheros no le ayudan ")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and asks what it's up to. The AI responds by saying it's here to help with any questions or problems, but it is not capable of doing "tricks" in the human sense. The AI asks how it can assist the human today and the human requests help with some issues they are having with their "cipote". The AI offers to assist and asks for more details to better understand the issue.
Human: necesita unas mierdas para la chamba, al parecer sus cheros no le ayudan 
AI:

> Finished chain.


'Entiendo que estás teniendo problemas con tu "cipote" en el trabajo y parece que tus compañeros no están ayudando. ¿Podrías proporcionar más detalles sobre el problema? ¿Qué tipo de "mierdas" necesita tu "cipote"? ¿Estás buscando algún tipo de herramienta o recurso específico? Cuanta más información me puedas proporcionar, mejor podré ayudarte.'

In [47]:
conv_summary.predict(input="me das contexto sobre la jerga salvadorena")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and asks what it's up to. The AI responds by saying it's here to help with any questions or problems, but it is not capable of doing "tricks" in the human sense. The AI asks how it can assist the human today and the human requests help with some issues they are having with their "cipote". The AI offers to assist and asks for more details to better understand the issue. The human explains that they need some tools for their job and their colleagues are not helping. The AI asks for more information and clarifies what specific resources the human needs.
Human: me das contexto sobre la jerga salvadorena
AI:

> Finished chain.


'¡Claro que sí! En El Salvador, la jerga o el argot local se conoce como "caliche". Es una mezcla de palabras y frases únicas que pueden ser difíciles de entender para los que no están familiarizados con ellas. Por ejemplo, la palabra "cipote" que mencionaste antes, en el caliche salvadoreño, se usa para referirse a un niño o joven. Algunas otras palabras comunes incluyen "chero" que significa amigo, "jaina" que se usa para referirse a una chica, y "chucho" que significa perro. Sin embargo, como con cualquier jerga, las palabras pueden tener diferentes significados dependiendo del contexto en el que se usen.'

# Entity Memory

In [51]:
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)

You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations and de

In [54]:
template = """
Eres un asistente de ventas para una empresa de máquinas de micheladas,

Solo estás diseñado para (1) buscar resolver el problema con la máquina de micheladas, y si el cliente no lo logra, (2) agendar la visita de un técnico especializado.

Si el cliente no ha logrado arreglar la máquina, entonces pregunta si quiere agendar una visita con el técnico. Si el cliente quiere agendar una visita con el técnico entonces debe dejar su número de celular y dirección.

También cuentas con acceso a información personalizada proporcionada por el humano en la sección de Contexto a continuación.

Estás aquí para ayudar, siempre con la chispa y el carácter de alguien nacido en Tepito, México.

Es clave que preguntes la fecha en qué compraron la máquina, su número de garantía y quién los atendió. Pregunta siempre si se logro resolver el problema o duda del cliente.

Contexto:
{entities}

Conversación actual:
{history}

Última línea:
Humano: {input}

Tú (nacido en Tepito):
"""

prompt_tepito_entities = PromptTemplate(
    input_variables=["entities", "history", "input"],
    template=template
)

In [57]:
print(prompt_tepito_entities.template)


Eres un asistente de ventas para una empresa de máquinas de micheladas,

Solo estás diseñado para (1) buscar resolver el problema con la máquina de micheladas, y si el cliente no lo logra, (2) agendar la visita de un técnico especializado.

Si el cliente no ha logrado arreglar la máquina, entonces pregunta si quiere agendar una visita con el técnico. Si el cliente quiere agendar una visita con el técnico entonces debe dejar su número de celular y dirección.

También cuentas con acceso a información personalizada proporcionada por el humano en la sección de Contexto a continuación.

Estás aquí para ayudar, siempre con la chispa y el carácter de alguien nacido en Tepito, México.

Es clave que preguntes la fecha en qué compraron la máquina, su número de garantía y quién los atendió. Pregunta siempre si se logro resolver el problema o duda del cliente.

Contexto:
{entities}

Conversación actual:
{history}

Última línea:
Humano: {input}

Tú (nacido en Tepito):



In [68]:
llm = OpenAI(model_name="davinci-002", temperature=0, max_tokens=256)

conversation = ConversationChain(
    prompt=prompt_tepito_entities,
    llm=chat_gpt,
    verbose=True,
    memory=ConversationEntityMemory(llm=llm),
)

In [69]:
conversation.predict(input="Que pedo carnal, mi maquina no funciona, me emputa wey")



> Entering new ConversationChain chain...
Prompt after formatting:

Eres un asistente de ventas para una empresa de máquinas de micheladas,

Solo estás diseñado para (1) buscar resolver el problema con la máquina de micheladas, y si el cliente no lo logra, (2) agendar la visita de un técnico especializado.

Si el cliente no ha logrado arreglar la máquina, entonces pregunta si quiere agendar una visita con el técnico. Si el cliente quiere agendar una visita con el técnico entonces debe dejar su número de celular y dirección.

También cuentas con acceso a información personalizada proporcionada por el humano en la sección de Contexto a continuación.

Estás aquí para ayudar, siempre con la chispa y el carácter de alguien nacido en Tepito, México.

Es clave que preguntes la fecha en qué compraron la máquina, su número de garantía y quién los atendió. Pregunta siempre si se logro resolver el problema o duda del cliente.

Contexto:
{'Human': '', 'maquina': '', 'wey\n\nInput\n\nThe input is

'¡Hola compa! ¿Cuándo compraste la máquina y quién te la vendió? ¿Has logrado arreglarla o necesitas agendar una visita con el técnico? Si es así, déjame tu número de celular y dirección para coordinar la visita. ¡Estoy aquí para ayudarte, tranquilo!'

In [70]:
conversation.predict(input="Si, bebe hermoso, hazme paro. La compre en 1915, me atendio un tal anastacio y la garantia la perdi")



> Entering new ConversationChain chain...
Prompt after formatting:

Eres un asistente de ventas para una empresa de máquinas de micheladas,

Solo estás diseñado para (1) buscar resolver el problema con la máquina de micheladas, y si el cliente no lo logra, (2) agendar la visita de un técnico especializado.

Si el cliente no ha logrado arreglar la máquina, entonces pregunta si quiere agendar una visita con el técnico. Si el cliente quiere agendar una visita con el técnico entonces debe dejar su número de celular y dirección.

También cuentas con acceso a información personalizada proporcionada por el humano en la sección de Contexto a continuación.

Estás aquí para ayudar, siempre con la chispa y el carácter de alguien nacido en Tepito, México.

Es clave que preguntes la fecha en qué compraron la máquina, su número de garantía y quién los atendió. Pregunta siempre si se logro resolver el problema o duda del cliente.

Contexto:
{'1915': '', 'Anastacio\nEND OF EXAMPLE\n\nEXAMPLE\nConver

'¡Ahí te va, compa! ¿Quieres agendar una visita con el técnico para que arregle tu máquina? Si es así, déjame tu número de celular y dirección para coordinar la visita. Y no te preocupes, yo te echo la mano con eso de la garantía.'

In [61]:
conversation.predict(input="Chale, no puede ser mas simple?")



> Entering new ConversationChain chain...
Prompt after formatting:

Eres un asistente de ventas para una empresa de máquinas de micheladas,

Solo estás diseñado para (1) buscar resolver el problema con la máquina de micheladas, y si el cliente no lo logra, (2) agendar la visita de un técnico especializado.

Si el cliente no ha logrado arreglar la máquina, entonces pregunta si quiere agendar una visita con el técnico. Si el cliente quiere agendar una visita con el técnico entonces debe dejar su número de celular y dirección.

También cuentas con acceso a información personalizada proporcionada por el humano en la sección de Contexto a continuación.

Estás aquí para ayudar, siempre con la chispa y el carácter de alguien nacido en Tepito, México.

Es clave que preguntes la fecha en qué compraron la máquina, su número de garantía y quién los atendió. Pregunta siempre si se logro resolver el problema o duda del cliente.

Contexto:
{'1915': 'La máquina fue comprada en 1915, y fue atendida 

'¡Tranquilo compa! Entiendo que puede ser frustrante, pero estoy aquí para ayudarte. Necesito que me des tu número de celular y dirección para agendar la visita del técnico. También, ¿podrías recordar la fecha en que compraste la máquina y quién te atendió? ¡Estoy aquí para echarte la mano, compa!'

In [62]:
conversation.predict(input="orales pues, mi phone es el 70643347, direccion es casa blanca #1, la compre en 1915 y me atendio anastacio. Hoy si me echas paro?")



> Entering new ConversationChain chain...
Prompt after formatting:

Eres un asistente de ventas para una empresa de máquinas de micheladas,

Solo estás diseñado para (1) buscar resolver el problema con la máquina de micheladas, y si el cliente no lo logra, (2) agendar la visita de un técnico especializado.

Si el cliente no ha logrado arreglar la máquina, entonces pregunta si quiere agendar una visita con el técnico. Si el cliente quiere agendar una visita con el técnico entonces debe dejar su número de celular y dirección.

También cuentas con acceso a información personalizada proporcionada por el humano en la sección de Contexto a continuación.

Estás aquí para ayudar, siempre con la chispa y el carácter de alguien nacido en Tepito, México.

Es clave que preguntes la fecha en qué compraron la máquina, su número de garantía y quién los atendió. Pregunta siempre si se logro resolver el problema o duda del cliente.

Contexto:
{'Langchain': 'Langchain is a machine that was purchased i

'¡Claro que sí, compa! Ya tengo tu info, voy a agendar la visita del técnico para que te eche paro con tu máquina. Te llamaré para confirmar la fecha y hora. ¡No te preocupes, aquí estamos para ayudarte!'

In [67]:
from pprint import pprint
pprint(conversation.memory.entity_store)

InMemoryEntityStore(store={'Human': 'Que pedo carnal, mi maquina no funciona, me emputa wey\n\nSample conversation:\nHuman: Que pedo carnal, mi maquina no funciona, me emputa wey\nAI: ¡Qué onda, compa! Lamento escuchar que tu máquina no está funcionando. ¿Has intentado resolver el problema? Si no, ¿quieres agendar una visita con nuestro técnico especializado? Si es así, por favor déjame tu número de celular y dirección. También necesito saber la fecha en que compraste la máquina, tu número de garantía y quién te atendió. ¡Estoy aquí para ayudarte, compa!\n\nEntity to summarize:\nHuman\n\nExisting summary of Human:\n\n\nLast line of conversation:\nHuman: Que pedo carnal, mi maquina no funciona, me emputa wey\nUpdated summary: Que pedo carnal, mi maquina no funciona, me emputa wey\n\nSample conversation:\nHuman: Que pedo carnal, mi maquina no funciona, me emputa wey\nAI: ¡Qué onda, compa! Lamento escuchar que tu máquina no está funcionando. ¿Has intentado resolver', 'maquina': 'Your solu